In [2]:
# need to install sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy.sql import text

In [3]:
sqlite = create_engine('sqlite:///lab1_bank.sqlite')
sqlite

Engine(sqlite:///lab1_bank.sqlite)

In [4]:
# Fetch all table names
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", sqlite)
print("Tables in the database:\n", tables)


Tables in the database:
        name
0   account
1      card
2    client
3      disp
4  district
5      loan
6     order
7     trans


Get the id values of the first 5 clients from district_id with a value equals to 1.

In [5]:
def run_query(query):
    with sqlite.connect() as conn:
        df = pd.read_sql_query(query, conn)
        return df

In [6]:
task_1 = run_query("select client_id from client where district_id = 1 limit 5;")
task_1

,client_id
0,2
1,3
2,22
3,23
4,28


In the client table, get an id value of the last client where the district_id equals to 72.

In [7]:
task_2 = run_query("""
select client_id from client where district_id = 72 order by client_id desc
limit 1;
""")
task_2

,client_id
0,13576


Get the 3 lowest amounts in the loan table


In [8]:
task_3 = run_query("""
select amount from loan
order by amount
limit 3;
                   """)

task_3

,amount
0,4980
1,5148
2,7656


What are the possible values for status, ordered alphabetically in ascending order in the loan table?

In [9]:
task_4 = run_query("""

select distinct status from loan
order by status;

""")
task_4

,status
0,A
1,B
2,C
3,D


What is the loan_id of the highest payment received in the loan table?



In [10]:
task_5  = run_query("""
select loan_id from loan
order by payments desc
limit 1;
""")
task_5

,loan_id
0,6415


What is the loan amount of the lowest 5 account_ids in the loan table? Show the account_id and the corresponding amount



In [14]:
task_6 = run_query("""
select account_id, amount from loan
order by loan_id 
limit 5;
""")
task_6

,account_id,amount
0,2,80952
1,19,30276
2,25,30276
3,37,318480
4,38,110736


What are the account_ids with the lowest loan amount that have a loan duration of 60 in the loan table?

In [19]:
task_7 = run_query("""
select account_id from loan
where duration = 60
order by amount
limit 5;
""")

task_7

,account_id
0,10954
1,938
2,10711
3,1766
4,10799


What are the unique values of k_symbol in the order table?

Note: There shouldn't be a table name order, since order is reserved from the ORDER BY clause. You have to use backticks to escape the order table name.

In [21]:
task_8 = run_query("""
select distinct k_symbol from `order`
where k_symbol != '' and k_symbol is not null;

""")
task_8

,k_symbol
0,SIPO
1,UVER
2,
3,POJISTNE
4,LEASING


In the order table, what are the order_ids of the client with the account_id 34?

In [26]:
task_9 = run_query("""
select order_id from `order`
where account_id = 34;
""")
task_9

,order_id
0,29445
1,29446
2,29447


In the order table, which account_ids were responsible for orders between order_id 29540 and order_id 29560 (inclusive)?

In [29]:
task_10 = run_query("""
select account_id from 'order'
where order_id between 29540 and 29560
group by account_id;
""")
task_10

,account_id
0,88
1,90
2,96
3,97


In the order table, what are the individual amounts that were sent to (account_to) id 30067122?

In [31]:
task_11 = run_query("""
select amount from 'order'
where account_to = 30067122;
""")

task_11

,amount
0,5123.0


In the trans table, show the trans_id, date, type and amount of the 10 first transactions from account_id 793 in chronological order, from newest to oldest.

In [35]:
task_12 = run_query("""
select trans_id, date, type, amount from trans
where account_id = 793
order by date desc
limit 10;
""")

task_12

,trans_id,date,type,amount
0,3556468,981231,PRIJEM,78.6
1,233254,981216,VYDAJ,600.0
2,233104,981212,VYDAJ,1212.0
3,233248,981211,VYDAJ,851.0
4,233176,981207,VYDAJ,204.0
5,3556467,981130,PRIJEM,75.1
6,233395,981130,VYDAJ,14.6
7,233103,981112,VYDAJ,1212.0
8,233247,981111,VYDAJ,851.0
9,233175,981107,VYDAJ,204.0


In the client table, of all districts with a district_id lower than 10, how many clients are from each district_id? Show the results sorted by the district_id in ascending order.

In [38]:
task_13 = run_query("""
select  count(client_id) from client
where district_id < 10
group by district_id;
""")

task_13

,count(client_id)
0,663
1,46
2,63
3,50
4,71
5,53
6,45
7,69
8,60


In the card table, how many cards exist for each type? Rank the result starting with the most frequent type.

In [41]:
task_14 = run_query("""
select type, count(*) from card
group by type
order by count(*) desc;

""")
task_14

,type,count(*)
0,classic,659
1,junior,145
2,gold,88


Using the loan table, print the top 10 account_ids based on the sum of all of their loan amounts.

In [42]:
task_15 = run_query("""
select account_id, amount from loan
group by account_id
order by sum(amount) desc
limit 10;

""")

task_15

,account_id,amount
0,7542,590820
1,8926,566640
2,2335,541200
3,817,538500
4,2936,504000
5,7049,495180
6,10451,482940
7,6950,475680
8,7966,473280
9,339,468060


In the loan table, retrieve the number of loans issued for each day, before (excl) 930907, ordered by date in descending order.

In [45]:
task_16 = run_query("""
select date, count(*) from loan
where date < 930907
group by date
order by date desc;
""")
task_16

,date,count(*)
0,930906,1
1,930803,1
2,930728,1
3,930711,1
4,930705,1


In the loan table, for each day in December 1997, count the number of loans issued for each unique loan duration, ordered by date and duration, both in ascending order. You can ignore days without any loans in your output.

In [54]:
task_17 = run_query("""
SELECT date, duration, COUNT(*) AS loan_count  
FROM loan  
WHERE date BETWEEN 971201 AND 971231  
GROUP BY date, duration 
ORDER BY date ASC, duration ASC;  
""")
task_17


,date,duration,loan_count
0,971206,24,1
1,971206,36,1
2,971208,12,3
3,971209,12,1
4,971209,24,1
5,971210,12,1
6,971211,24,1
7,971211,48,1
8,971213,24,1
9,971220,36,1


In the trans table, for account_id 396, sum the amount of transactions for each type (VYDAJ = Outgoing, PRIJEM = Incoming). Your output should have the account_id, the type and the sum of amount, named as total_amount. Sort alphabetically by type.

In [50]:
task_18 = run_query("""
select account_id, type, sum(amount) as total_amount from trans
where account_id = 396
group by account_id, type;

""")
task_18

,account_id,type,total_amount
0,396,PRIJEM,1028138.7
1,396,VYDAJ,1485814.4
